## Step 18 메모리 절약모드

In [16]:
import numpy as np
import weakref 

In [ ]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{}은(는) 지원하지 않습니다.'.format(type(data)))
            
        self.data = data
        self.grad = None 
        self.creator = None
        self.generation = 0 

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1 

    def backward(self, retain_grad = False): #retain_grad 변수 추가, True면 모든 변수가 미분값 유지, False면 중간 변수 미분값 모두 None
        if self.grad is None:
            self.grad = np.ones_like(self.data) 

        funcs = []
        seen_set = set() 

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)
        
        add_func(self.creator)#

        while funcs:
            f = funcs.pop() 
            gys = [output().grad for output in f.outputs]
            gxs = f.backward(*gys) 
            if not isinstance(gxs, tuple): 
                gxs = (gxs,)
            
            for x, gx in zip(f.inputs, gxs): 
                if x.grad is None: 
                    x.grad = gx
                else:
                    x.grad = x.grad + gx 

                if x.creator is not None:
                    add_func(x.creator)
            if not retain_grad: # retain_grad False면 중간변수의 미분값 모두 None
                for y in f.outputs:
                    y().grad = None # y는 weakref
    
    def cleargrad(self):
        self.grad = None 

In [18]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

In [19]:
class Function:
    def __call__(self, *inputs): 
        xs = [x.data for x in inputs] 
        ys = self.forward(*xs) 
        if not isinstance(ys, tuple): 
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys] 

        if Config.enable_backprop: #True일때만 역전파 코드 실행
            self.generation = max([x.generation for x in inputs]) 
            for output in outputs:
                output.set_creator(self) 
            self.inputs = inputs #추론시에는 순전파만 하기때문에 미분값 필요없음
            self.outputs = [weakref.ref(output) for output in outputs]
        return outputs if len(outputs) >1 else outputs[0] 
    
    def forward(self, xs):
        raise NotImplementedError()
    def backward(self, gys):
        raise NotImplementedError()

In [20]:
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y
    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx
def square(x):
    return Square()(x) 
    
class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx
def exp(x):
    return Exp()(x)

class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y
    
    def backward(self, gy):
        return gy, gy 
def add(x0, x1):
    return Add()(x0, x1)

In [21]:
class Config: # 설정데이터기 때문에 인스턴스화 하지 않고 클래스 상태로만 이용
    enable_backprop = True #True 면 역전파 활성

In [ ]:
import contextlib

@contextlib.contextmanager
def using_config(name, value): #name type: str, Config 속성의 이름
    old_value = getattr(Config, name)
    setattr(Config, name, value)
    try:
        yield
    finally:
        setattr(Config, name, old_value)

In [27]:
with using_config("enable_backprop", False): # with 안에서만 역전파 비활성 모드
    x = Variable(np.array(2.0))
    y = square(x)

In [ ]:
def no_grad(): #using_config함수 짧게
    return using_config('enable_backprop', False)

with no_grad():
    x = Variable(np.array(2.0))
    y = square(x)